In [1]:
import os
from scipy.io import wavfile
from pystoi import stoi
import librosa
import soundfile as sf
from pypesq import pesq
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import librosa.display
import IPython.display as ipd

In [2]:
model = tf.contrib.keras.models.load_model('mixed_pos_snr_STFT_GRUx3_tanh_5epoch.h5')

W0725 09:01:09.328017 140341936031232 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
#抓取路徑
def get_filepaths(directory):
    """
    This function will generate the file names in a directory 
    tree by walking the tree either top-down or bottom-up. For each 
    directory in the tree rooted at directory top (including top itself), 
    it yields a 3-tuple (dirpath, dirnames, filenames).
    """
    file_paths = []  # List which will store all of the full filepaths.

    # Walk the tree.

    for root, directories, files in os.walk(directory):

        for filename in files:
            if filename.endswith('.wav'):
            # Join the two strings in order to form the full filepath.
                filepath = os.path.join(root, filename)
                file_paths.append(filepath)  # Add it to the list.
                # pdb.set_trace()

    return file_paths  # Self-explanatory.
mixed_file=get_filepaths('mixed_pos_snr/')
cleaned_file=get_filepaths('clean/')

In [4]:
#整理檔案路徑排序
#確認乾淨的答案有對應到混音完的檔案
mixed_file = sorted(get_filepaths('mixed_pos_snr/'))
clean_file = sorted(get_filepaths('clean/'))

cleaned_file = []
for data in clean_file:
    for i in range(10):
        cleaned_file += [data]
        i += 1


len(mixed_file), len(cleaned_file), mixed_file[5410:5420], cleaned_file[5410:5420]    # 確定排序  

(64000,
 64000,
 ['mixed_pos_snr/amanda_T23-1_20200626_ambulance_passing_snr12.wav',
  'mixed_pos_snr/amanda_T23-1_20200626_breaking_concrete_snr4.wav',
  'mixed_pos_snr/amanda_T23-1_20200626_breaking_houses2_snr11.wav',
  'mixed_pos_snr/amanda_T23-1_20200626_breaking_houses4_snr9.wav',
  'mixed_pos_snr/amanda_T23-1_20200626_car_road_snr13.wav',
  'mixed_pos_snr/amanda_T23-1_20200626_construction2_snr13.wav',
  'mixed_pos_snr/amanda_T23-1_20200626_fireengine_starting_snr9.wav',
  'mixed_pos_snr/amanda_T23-1_20200626_house_construction_snr7.wav',
  'mixed_pos_snr/amanda_T23-1_20200626_trains_passing1_snr9.wav',
  'mixed_pos_snr/amanda_T23-1_20200626_trains_passing4_snr3.wav'],
 ['clean/amanda/amanda_T23-1_20200626.wav',
  'clean/amanda/amanda_T23-1_20200626.wav',
  'clean/amanda/amanda_T23-1_20200626.wav',
  'clean/amanda/amanda_T23-1_20200626.wav',
  'clean/amanda/amanda_T23-1_20200626.wav',
  'clean/amanda/amanda_T23-1_20200626.wav',
  'clean/amanda/amanda_T23-1_20200626.wav',
  'clea

In [5]:
#找出最長的音檔(88800),並變成相同長度
max_len=0
clean_filename=''
for i in range(len(cleaned_file)):
    path = cleaned_file[i]
    noisy, rate  = librosa.load(path,sr=16000) 
    if noisy.shape[0]>max_len:
        max_len=noisy.shape[0]
        clean_filename = path

In [6]:
max_len,clean_filename

(85600, 'clean/saphira/saphira_T19-2_20200626.wav')

In [10]:
stoi_list = []
pesq_list = []
l=0
for path in mixed_file[:16000]:
    noisy, rate = librosa.load(path, sr=16000)
    noisy = noisy.astype('float32')
    noisy = np.hstack((noisy, np.zeros(85600 - noisy.shape[0])))
    noisy_stft = librosa.stft(noisy,n_fft=512,hop_length=256,win_length=512,center=False)
   
    n_real = np.abs(noisy_stft)
    n_real = n_real.T
    n_phase = np.angle(noisy_stft)
    n_real = n_real.reshape(1, n_real.shape[0], n_real.shape[1])
 
    enhanced = model.predict(n_real)
    enhanced = np.squeeze(enhanced).T
    Rec = np.multiply(enhanced, np.exp(1j * n_phase))
    
    Rec = librosa.istft(Rec, hop_length=256, win_length=512, center=False)
    Rec[100:-100] *= 50
    Rec = (Rec - np.mean(Rec)) / np.std(Rec)
    Rec = np.hstack((Rec, np.zeros(85600 - Rec.shape[0])))
    
    clean, sr = librosa.load(cleaned_file[l], sr=16000)
    clean = np.hstack((clean, np.zeros(85600 - clean.shape[0])))
   
#     Clean and den should have the same length, and be 1D
    pt_stoi = stoi(clean, Rec, 16000, extended=False)
    stoi_list += [pt_stoi]
    pt_pesq = pesq(clean, Rec, 16000)
    pesq_list += [pt_pesq]
    l += 1
   
max(stoi_list), max(pesq_list)


(0.9782665530986326, 3.3639090061187744)

In [12]:
stoi_list01 = []
pesq_list01 = []
l=16000
for path in mixed_file[16000:32000]:
    noisy, rate = librosa.load(path, sr=16000)
    noisy = noisy.astype('float32')
    noisy = np.hstack((noisy, np.zeros(85600 - noisy.shape[0])))
    noisy_stft = librosa.stft(noisy,n_fft=512,hop_length=256,win_length=512,center=False)
   
    n_real = np.abs(noisy_stft)
    n_real = n_real.T
    n_phase = np.angle(noisy_stft)
    n_real = n_real.reshape(1, n_real.shape[0], n_real.shape[1])
 
    enhanced = model.predict(n_real)
    enhanced = np.squeeze(enhanced).T
    Rec = np.multiply(enhanced, np.exp(1j * n_phase))
    
    Rec = librosa.istft(Rec, hop_length=256, win_length=512, center=False)
    Rec[100:-100] *= 50
    Rec = (Rec - np.mean(Rec)) / np.std(Rec)
    Rec = np.hstack((Rec, np.zeros(85600 - Rec.shape[0])))
    
    clean, sr = librosa.load(cleaned_file[l], sr=16000)
    clean = np.hstack((clean, np.zeros(85600 - clean.shape[0])))
   
#     Clean and den should have the same length, and be 1D
    pt_stoi = stoi(clean, Rec, 16000, extended=False)
    stoi_list01 += [pt_stoi]
    pt_pesq = pesq(clean, Rec, 16000)
    pesq_list01 += [pt_pesq]
    l += 1
   
max(stoi_list01), max(pesq_list01)


(0.9277239362494457, 3.6452934741973877)

In [13]:
stoi_list02 = []
pesq_list02 = []
l=32000
for path in mixed_file[32000:48000]:
    noisy, rate = librosa.load(path, sr=16000)
    noisy = noisy.astype('float32')
    noisy = np.hstack((noisy, np.zeros(85600 - noisy.shape[0])))
    noisy_stft = librosa.stft(noisy,n_fft=512,hop_length=256,win_length=512,center=False)
   
    n_real = np.abs(noisy_stft)
    n_real = n_real.T
    n_phase = np.angle(noisy_stft)
    n_real = n_real.reshape(1, n_real.shape[0], n_real.shape[1])
 
    enhanced = model.predict(n_real)
    enhanced = np.squeeze(enhanced).T
    Rec = np.multiply(enhanced, np.exp(1j * n_phase))
    
    Rec = librosa.istft(Rec, hop_length=256, win_length=512, center=False)
    Rec[100:-100] *= 50
    Rec = (Rec - np.mean(Rec)) / np.std(Rec)
    Rec = np.hstack((Rec, np.zeros(85600 - Rec.shape[0])))
    
    clean, sr = librosa.load(cleaned_file[l], sr=16000)
    clean = np.hstack((clean, np.zeros(85600 - clean.shape[0])))
   
#     Clean and den should have the same length, and be 1D
    pt_stoi = stoi(clean, Rec, 16000, extended=False)
    stoi_list02 += [pt_stoi]
    pt_pesq = pesq(clean, Rec, 16000)
    pesq_list02 += [pt_pesq]
    l += 1
   
max(stoi_list02), max(pesq_list02)

(0.9441466670695038, 3.4864654541015625)

In [17]:
stoi_list03 = []
pesq_list03 = []
l=48000
for path in mixed_file[48000:64000]:
    noisy, rate = librosa.load(path, sr=16000)
    noisy = noisy.astype('float32')
    noisy = np.hstack((noisy, np.zeros(85600 - noisy.shape[0])))
    noisy_stft = librosa.stft(noisy,n_fft=512,hop_length=256,win_length=512,center=False)
   
    n_real = np.abs(noisy_stft)
    n_real = n_real.T
    n_phase = np.angle(noisy_stft)
    n_real = n_real.reshape(1, n_real.shape[0], n_real.shape[1])
 
    enhanced = model.predict(n_real)
    enhanced = np.squeeze(enhanced).T
    Rec = np.multiply(enhanced, np.exp(1j * n_phase))
    
    Rec = librosa.istft(Rec, hop_length=256, win_length=512, center=False)
    Rec[100:-100] *= 50
    Rec = (Rec - np.mean(Rec)) / np.std(Rec)
    Rec = np.hstack((Rec, np.zeros(85600 - Rec.shape[0])))
    
    clean, sr = librosa.load(cleaned_file[l], sr=16000)
    clean = np.hstack((clean, np.zeros(85600 - clean.shape[0])))
   
#     Clean and den should have the same length, and be 1D
    pt_stoi = stoi(clean, Rec, 16000, extended=False)
    stoi_list03 += [pt_stoi] 
    pt_pesq = pesq(clean, Rec, 16000)
    pesq_list03 += [pt_pesq] 
    l += 1
   
max(stoi_list03), max(pesq_list03)

(0.9724147990125118, 3.364743709564209)

In [59]:
tot_stoi = (stoi_list + stoi_list01 + stoi_list02 + stoi_list03)
tot_pesq = (pesq_list + pesq_list01 + pesq_list02 + pesq_list03)

In [62]:
sor_tot_stoi = sorted(tot_stoi)
sor_tot_pesq = sorted(tot_pesq)

In [63]:
np.argmax(np.array(tot_pesq))

46750

In [64]:
mixed_file[46750]

'mixed_pos_snr/ryan_T20-5_20200620_ambulance_passing_snr14.wav'

In [74]:
score_lst=[]
name_lst=[]
for i in range(1,6):
    max1 = tot_stoi.index(sor_tot_stoi[-i])
    name_lst.append(mixed_file[max1])
    score_lst.append(tot_stoi[max1])
score_lst,name_lst


([0.9782665530986326,
  0.9775086680408039,
  0.9774873002722295,
  0.976089331391819,
  0.9758601787479622],
 ['mixed_pos_snr/blue_T27-10_20200710_fireengine_starting_snr14.wav',
  'mixed_pos_snr/blue_T27-10_20200710_ambulance_passing_snr2.wav',
  'mixed_pos_snr/blue_T04-2_20200710_fireengine_starting_snr15.wav',
  'mixed_pos_snr/blue_T26-10_20200710_ambulance_passing_snr15.wav',
  'mixed_pos_snr/blue_T10-2_20200710_fireengine_starting_snr14.wav'])

In [76]:
score_lst1=[]
name_lst1=[]
for i in range(1,6):
    max2 = tot_pesq.index(sor_tot_pesq[-i])
    name_lst1.append(mixed_file[max2])
    score_lst1.append(tot_pesq[max2])
score_lst1,name_lst1


([3.6452934741973877,
  3.608628273010254,
  3.5849177837371826,
  3.5654938220977783,
  3.554208755493164],
 ['mixed_pos_snr/ryan_T20-5_20200620_ambulance_passing_snr14.wav',
  'mixed_pos_snr/ryan_T04-6_20200620_ambulance_passing_snr12.wav',
  'mixed_pos_snr/ryan_T28-6_20200620_ambulance_passing_snr6.wav',
  'mixed_pos_snr/ryan_T19-10_20200620_ambulance_passing_snr14.wav',
  'mixed_pos_snr/ryan_T04-8_20200620_ambulance_passing_snr2.wav'])

In [71]:
tot_stoi[max1]

0.9782665530986326

In [67]:
print(f'first stoi : {sor_tot_stoi[-1]}'),
print(f'second stoi : {sor_tot_stoi[-2]}'),
print(f'third stoi : {sor_tot_stoi[-3]}')

first stoi : 0.9782665530986326
second stoi : 0.9775086680408039
third stoi : 0.9774873002722295


In [68]:
print(f'first pesq : {sor_tot_pesq[-1]}'),
print(f'second pesq : {sor_tot_pesq[-2]}'),
print(f'third pesq : {sor_tot_pesq[-3]}')

first pesq : 3.6452934741973877
second pesq : 3.608628273010254
third pesq : 3.5849177837371826


In [ ]:
GRU

In [7]:
print(f'PESQ : 3.6452934741973877 , name : ryan_T20-5_20200620_ambulance_passing_snr14.wav'),
print(f'PESQ : 3.6086282730102546 , name : ryan_T04-6_20200620_ambulance_passing_snr12.wav'),
print(f'PESQ : 3.5849177837371826 , name : ryan_T28-6_20200620_ambulance_passing_snr6.wav')

PESQ : 3.6452934741973877 , name : ryan_T20-5_20200620_ambulance_passing_snr14.wav
PESQ : 3.6086282730102546 , name : ryan_T04-6_20200620_ambulance_passing_snr12.wav
PESQ : 3.5849177837371826 , name : ryan_T28-6_20200620_ambulance_passing_snr6.wav


In [3]:
print(f'STOI : 0.9782665530986326 , name : blue_T27-10_20200710_fireengine_starting_snr14.wav'),
print(f'STOI : 0.9775086680408039 , name : blue_T27-10_20200710_ambulance_passing_snr2.wav'),
print(f'STOI : 0.9774873002722295 , name : blue_T04-2_20200710_fireengine_starting_snr15.wav')

STOI : 0.9782665530986326 , name : blue_T27-10_20200710_fireengine_starting_snr14.wav
STOI : 0.9775086680408039 , name : blue_T27-10_20200710_ambulance_passing_snr2.wav
STOI : 0.9774873002722295 , name : blue_T04-2_20200710_fireengine_starting_snr15.wav


In [ ]:
CONV2D

In [5]:
print(f'PESQ : 3.8121867179870605 , name : saphira_T13-9_20200626_ambulance_passing_snr13.wav'),
print(f'PESQ : 3.8036730289459232 , name : blue_T03-8_20200710_ambulance_passing_snr5.wav'),
print(f'PESQ : 3.7971813678741455 , name : jin_T18-2_20200620_ambulance_passing_snr12.wav')

PESQ : 3.8121867179870605 , name : saphira_T13-9_20200626_ambulance_passing_snr13.wav
PESQ : 3.8036730289459232 , name : blue_T03-8_20200710_ambulance_passing_snr5.wav
PESQ : 3.7971813678741455 , name : jin_T18-2_20200620_ambulance_passing_snr12.wav


In [6]:
print(f'STOI : 0.9909236116918912 , name : blue_T19-3_20200710_ambulance_passing_snr14.wav'),
print(f'STOI : 0.9901033438609617 , name : blue_T27-10_20200710_ambulance_passing_snr8.wav'),
print(f'STOI : 0.9900976538725138 , name : blue_T20-2_20200710_ambulance_passing_snr14.wav')

STOI : 0.9909236116918912 , name : blue_T19-3_20200710_ambulance_passing_snr14.wav
STOI : 0.9901033438609617 , name : blue_T27-10_20200710_ambulance_passing_snr8.wav
STOI : 0.9900976538725138 , name : blue_T20-2_20200710_ambulance_passing_snr14.wav


In [ ]:
#stoi 評分

clean, fs = sf.read('path/to/clean/audio')
denoised, fs = sf.read('path/to/denoised/audio')

# Clean and den should have the same length, and be 1D
d = stoi(clean, denoised, fs, extended=False)

In [ ]:
from pystoi import stoi
from pypesq import pesq


i = 0
stoi_list = []
pesq_list = []
for path in mixed_file:
    noisy, rate = librosa.load(path, sr=16000)
    noisy = noisy.astype('float32')
    noisy = np.hstack((noisy, np.zeros(85600 - noisy.shape[0])))
    noisy_stft = librosa.stft(noisy,n_fft=512,hop_length=256,win_length=512,center=False)

    n_real = np.abs(noisy_stft)
    n_phase = np.angle(noisy_stft)
    n_real = n_real.reshape(1, n_real.shape[0], n_real.shape[1], 1)

    model._make_predict_function()

    enhanced = model.predict(n_real)
    enhanced = np.squeeze(enhanced)
    Rec = np.multiply(enhanced, np.exp(1j * n_phase))

    Rec = librosa.istft(Rec, hop_length=256, win_length=512, center=False)
    Rec[100:-100] *= 50
    Rec = (Rec - np.mean(Rec)) / np.std(Rec)
    
    clean, sr = librosa.load(cleaned_file[i], sr=16000)
    clean = np.hstack((clean, np.zeros(85504 - clean.shape[0])))
    
    # Clean and den should have the same length, and be 1D
    pt_stoi = stoi(clean, Rec, 16000, extended=False)
    stoi_list += [pt_stoi]
    pt_pesq = pesq(clean, Rec, 16000)
    pesq_list += [pt_pesq]
    i += 1

max(stoi_list), max(pesq_list)